# Introduction

Build a recommender system to identify top 5 recommended products for a user

## High Level Steps to be followed

| **Step** | **Notebook** |
|---|---|
| Data Acquisition | Implemented in Prior Notebook SBPRS_01_EDA_and_Recommendation |
| Data Cleansing | Implemented in Prior Notebook SBPRS_01_EDA_and_Recommendation |
| Basic EDA to understand the dataset | Implemented in Prior Notebook SBPRS_01_EDA_and_Recommendation |
| Preprocessing if any | Implemented in Prior Notebook SBPRS_01_EDA_and_Recommendation |
| Identify suitable Recommendation System | Implemented in Prior Notebook SBPRS_01_EDA_and_Recommendation |
| Identify suitable Classifier to predict sentiment | Implemented in Prior Notebook (SBPRS_02_Sentiment_Classifier) |
| **Identify top 20 Recommended Product for each user based on similarity score** | **Implemented in Current Notebook ( SBPRS_03_Consolidation )** |
| **For each review , Calculate Percentage of Postive Sentiment** | **Implemented in Current Notebook ( SBPRS_03_Consolidation )** |
| **Pick 5 Products which has top Recommendation Percentage** | **Implemented in Current Notebook ( SBPRS_03_Consolidation )**  |




NOTE : This notebook is executed on local Environment

Upload the following pickled files generated on previous step to models folder 

* Logistic_Reg_final_model.pkl
* cleansed_dataset_combined.pkl
* vectorizer.pkl
* full_user_final_rating.pkl


## Import Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import pickle 
import os
import gzip
import sklearn

## Read necessary datasets from pickle files

In [2]:
os.getcwd()

'C:\\Users\\dmanue11\\SBPRS\\notebooks'

In [3]:
pickle_file_path= "\\..\\models\\"


In [4]:
pkl_filename=os.getcwd()+pickle_file_path  +"Logistic_Reg_final_model.pkl"

with open(pkl_filename, 'rb') as file:
    lr_model= pickle.load(file)

In [5]:
pkl_filename=os.getcwd()+pickle_file_path  +"cleansed_dataset_combined.pkl"

with open(pkl_filename, 'rb') as file:
    cleansed_dataset_combined= pickle.load(file)

In [6]:
cleansed_dataset_combined.head(2).T

,0,1
id,AV13O1A8GV-KLJ3akUyj,AV14LG0R-jtxr-f38QfS
name,Pink Friday: Roman Reloaded Re-Up (w/dvd),Lundberg Organic Cinnamon Toast Rice Cakes
reviews_username,joshua,dorothy w
reviews_text,i love this album. it's very good. more to the...,Good flavor. This review was collected as part...
reviews_title,Just Awesome,Good
updated_sentiment,Positive,Positive
reviews_rating,5,5
normalised_rating,5,5
combined_title_text,Just Awesome . i love this album. it's very go...,Good . Good flavor. This review was collected ...
len_review,44,13


In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


import nltk
from nltk.tokenize import word_tokenize,TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
import string


In [8]:
pkl_filename=os.getcwd()+pickle_file_path  +"vectorizer.pkl"

with open(pkl_filename, 'rb') as file:
    tfidf_data= pickle.load(file)

In [9]:
#pkl_filename=os.getcwd()+pickle_file_path  +"full_user_final_rating.pkl"

#with open(pkl_filename, 'rb') as file:
#    full_user_final_rating= pickle.load(file)

In [10]:

pkl_filename=os.getcwd()+pickle_file_path  +"full_user_final_rating.data"

# read the file
fp = gzip.open(pkl_filename,'rb') #This assumes that full_user_final_rating.data is already packed with gzip
full_user_final_rating = pickle.load(fp)
fp.close()



## Predict the Sentiment for Review text 

In [11]:
cleansed_dataset_combined["predicted_sentiment"]=lr_model.predict(tfidf_data)

In [12]:
cleansed_dataset_combined["predicted_sentiment_text"]=cleansed_dataset_combined["predicted_sentiment"].map(lambda x : "Positive" if x==1 else "Negative")

## Calculate Positive Review Percentage for each Product

In [13]:
product_positive_total_reviews=cleansed_dataset_combined.groupby("name").agg({'predicted_sentiment': ['sum','count']})                              

In [14]:
product_positive_total_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271 entries, 0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest to Yes To Grapefruit Rejuvenating Body Wash
Data columns (total 2 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   (predicted_sentiment, sum)    271 non-null    int64
 1   (predicted_sentiment, count)  271 non-null    int64
dtypes: int64(2)
memory usage: 6.4+ KB


In [15]:
product_positive_total_reviews.columns=["_".join(x) for x in product_positive_total_reviews.columns.ravel()]

C:\Users\dmanue11\AppData\Local\Temp/ipykernel_12648/880418004.py:1: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  product_positive_total_reviews.columns=["_".join(x) for x in product_positive_total_reviews.columns.ravel()]


In [16]:
product_positive_total_reviews["pct_positive_review"]=product_positive_total_reviews.predicted_sentiment_sum/product_positive_total_reviews.predicted_sentiment_count*100

In [17]:
product_positive_total_reviews

,predicted_sentiment_sum,predicted_sentiment_count,pct_positive_review
name,,,
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,5,6,83.333333
100:Complete First Season (blu-Ray),130,139,93.525180
2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,1,4,25.000000
"2x Ultra Era with Oxi Booster, 50fl oz",5,5,100.000000
"42 Dual Drop Leaf Table with 2 Madrid Chairs""",1,1,100.000000
...,...,...,...
Weleda Everon Lip Balm,8,8,100.000000
Wilton Black Dots Standard Baking Cups,1,1,100.000000
Windex Original Glass Cleaner Refill 67.6oz (2 Liter),281,348,80.747126


In [18]:
full_user_final_rating.loc["repster1988"].sort_values(ascending=False)[0:20]

name
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest                          0.0
Pleasant Hearth 7.5 Steel Grate, 30 5 Bar - Black                                      0.0
Olivella Bar Soap - 3.52 Oz                                                            0.0
Orajel Maximum Strength Toothache Pain Relief Liquid                                   0.0
Pacific Natural Foods Organic Beef Broth                                               0.0
Pantene Color Preserve Volume Shampoo, 25.4oz                                          0.0
Pantene Pro-V Expert Collection Age Defy Conditioner                                   0.0
Pearhead Id Bracelet Frame                                                             0.0
Pendaflex174 Divide It Up File Folder, Multi Section, Letter, Assorted, 12/pack        0.0
Physicians Formula Mineral Wear Talc-Free Mineral Correcting Powder, Creamy Natural    0.0
Physicians Formula Powder Palette Mineral Glow Pearls, Translucent Pearl             

In [19]:
product_positive_total_reviews.reset_index()

,name,predicted_sentiment_sum,predicted_sentiment_count,pct_positive_review
0,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,5,6,83.333333
1,100:Complete First Season (blu-Ray),130,139,93.525180
2,2017-2018 Brownline174 Duraflex 14-Month Plann...,1,4,25.000000
3,"2x Ultra Era with Oxi Booster, 50fl oz",5,5,100.000000
4,"42 Dual Drop Leaf Table with 2 Madrid Chairs""",1,1,100.000000
...,...,...,...,...
266,Weleda Everon Lip Balm,8,8,100.000000
267,Wilton Black Dots Standard Baking Cups,1,1,100.000000
268,Windex Original Glass Cleaner Refill 67.6oz (2...,281,348,80.747126
269,Yes To Carrots Nourishing Body Wash,62,63,98.412698


## Calculate Top 5 Postive Products for each User

Logic to derive:
* For each User
    * Get Top 20 Recommended Products
    * For each product
        * Get Sentiment for all review
        * Calculate Percentage of Postive Sentiment for each product
    * Retain Top 5 products with positive review sentimeent   
    

In [20]:
final_user_df=pd.DataFrame()
for user in full_user_final_rating.index:
    print("Calculating for User - ", user)
    _top_20_products=full_user_final_rating.loc[user].sort_values(ascending=False)[0:20].reset_index()
    _top_20_products.columns=["product_name","recommended_pct"]
    _top_20_products=_top_20_products[_top_20_products.recommended_pct>0]
    _user_top5=pd.merge(_top_20_products, product_positive_total_reviews, how='left', left_on='product_name', right_on='name').sort_values(by="pct_positive_review",ascending=False)[0:5]
    _user_top5["username"]=user
    final_user_df=pd.concat([ _user_top5,final_user_df] )

Calculating for User -  00dog3
Calculating for User -  00sab00
Calculating for User -  01impala
Calculating for User -  02dakota
Calculating for User -  02deuce
Calculating for User -  0325home
Calculating for User -  06stidriver
Calculating for User -  08dallas
Calculating for User -  09mommy11
Calculating for User -  1.11E+24
Calculating for User -  1085
Calculating for User -  10ten
Calculating for User -  11111111aaaaaaaaaaaaaaaaa
Calculating for User -  1143mom
Calculating for User -  11677j
Calculating for User -  1234
Calculating for User -  1234561
Calculating for User -  1234567
Calculating for User -  1234asdf
Calculating for User -  123cat123
Calculating for User -  123charlie
Calculating for User -  123numbers
Calculating for User -  123rs
Calculating for User -  123soccermom
Calculating for User -  123too
Calculating for User -  127726
Calculating for User -  12cass12
Calculating for User -  12gage
Calculating for User -  132457
Calculating for User -  13dani
Calculating f

Calculating for User -  911notimetoshop
Calculating for User -  9373weld
Calculating for User -  987654321abc
Calculating for User -  98nd500
Calculating for User -  99runner
Calculating for User -  9thdoctor
Calculating for User -  _christenak
Calculating for User -  a hutch
Calculating for User -  a leaf on the family tree
Calculating for User -  a1126
Calculating for User -  a11mxdup
Calculating for User -  a1cbee1481
Calculating for User -  a1s3
Calculating for User -  a77077
Calculating for User -  a786
Calculating for User -  a7xmaddlove
Calculating for User -  aaaa
Calculating for User -  aaam
Calculating for User -  aabrannon
Calculating for User -  aac06002
Calculating for User -  aac3
Calculating for User -  aagbaghagmom
Calculating for User -  aakon
Calculating for User -  aalyah
Calculating for User -  aarhus74
Calculating for User -  aaron
Calculating for User -  aaronabigail
Calculating for User -  aaronlab
Calculating for User -  aaronm
Calculating for User -  aaube1990


Calculating for User -  ah87
Calculating for User -  aharbin88
Calculating for User -  aharry1021
Calculating for User -  ahart04
Calculating for User -  ahaschart
Calculating for User -  ahawk
Calculating for User -  aheadfullofdreams
Calculating for User -  ahelke
Calculating for User -  ahjohnson1983
Calculating for User -  ahmedsat
Calculating for User -  ahoy
Calculating for User -  ai85
Calculating for User -  aidensmama 88
Calculating for User -  aidnel646
Calculating for User -  aiihcy
Calculating for User -  aileen2
Calculating for User -  aim1
Calculating for User -  aimcoop
Calculating for User -  aimee
Calculating for User -  aira
Calculating for User -  airborne2
Calculating for User -  airborneannie
Calculating for User -  airforce65
Calculating for User -  airkingchris
Calculating for User -  airsteve
Calculating for User -  airwave15
Calculating for User -  aiscoupon
Calculating for User -  aisha
Calculating for User -  aiyana84
Calculating for User -  aj
Calculating fo

Calculating for User -  ally1989
Calculating for User -  allycat
Calculating for User -  allyg
Calculating for User -  allyj1984
Calculating for User -  allylapew
Calculating for User -  allymolaley
Calculating for User -  allyoueat
Calculating for User -  allyson
Calculating for User -  allyson55
Calculating for User -  allyv7
Calculating for User -  alm27
Calculating for User -  alma
Calculating for User -  alma82
Calculating for User -  almag21
Calculating for User -  almighty
Calculating for User -  almita84
Calculating for User -  alms816
Calculating for User -  alnscoob97
Calculating for User -  alo1991
Calculating for User -  aloha4u
Calculating for User -  alohaguy
Calculating for User -  alohalk
Calculating for User -  alohatiff
Calculating for User -  alondrasnoopy
Calculating for User -  aloras
Calculating for User -  alotaboutlivin
Calculating for User -  alowai
Calculating for User -  alowther
Calculating for User -  alpaben
Calculating for User -  alpat
Calculating for Us

Calculating for User -  anarosa
Calculating for User -  anas
Calculating for User -  anas79
Calculating for User -  anastasiya
Calculating for User -  anatnas4
Calculating for User -  anaws10
Calculating for User -  anaya5
Calculating for User -  anayajhene
Calculating for User -  ancatherman
Calculating for User -  andabear
Calculating for User -  ande
Calculating for User -  andi
Calculating for User -  andi78
Calculating for User -  andie
Calculating for User -  andie10
Calculating for User -  andie413
Calculating for User -  andieg
Calculating for User -  andiejo
Calculating for User -  andim26
Calculating for User -  andiz
Calculating for User -  andonopolus
Calculating for User -  andre2004
Calculating for User -  andrea
Calculating for User -  andrea0211
Calculating for User -  andrea4
Calculating for User -  andrea62
Calculating for User -  andreadm
Calculating for User -  andreafl
Calculating for User -  andreag
Calculating for User -  andreagarr
Calculating for User -  andrea

...

Calculating for User -  urbabyhrt
Calculating for User -  urbanchich
Calculating for User -  urbanguy
Calculating for User -  urbina2k9
Calculating for User -  urgirlvern
Calculating for User -  urkell
Calculating for User -  uronetwo
Calculating for User -  usachris2000
Calculating for User -  usaf
Calculating for User -  usaf1
Calculating for User -  usagirl05
Calculating for User -  usarmy8989
Calculating for User -  usc1
Calculating for User -  uscguy
Calculating for User -  uscruisers
Calculating for User -  usdun
Calculating for User -  usedtobeabeachboy
Calculating for User -  usedtolovenexxus
Calculating for User -  user
Calculating for User -  user4848
Calculating for User -  usforme
Calculating for User -  usmarine71
Calculating for User -  usmc611
Calculating for User -  utdao
Calculating for User -  utdawson
Calculating for User -  utpal
Calculating for User -  utsamom
Calculating for User -  utvolsfan
Calculating for User -  uu1622
Calculating for User -  v1988
Calculating

Calculating for User -  vlbailey
Calculating for User -  vllybll482
Calculating for User -  vlo910
Calculating for User -  vmax120
Calculating for User -  vmo24
Calculating for User -  vmorales
Calculating for User -  vmovie
Calculating for User -  vnss
Calculating for User -  voided
Calculating for User -  volivier68
Calculating for User -  volleyball21
Calculating for User -  vollink
Calculating for User -  voltagirl
Calculating for User -  voninky
Calculating for User -  vonn
Calculating for User -  vonna
Calculating for User -  vonny
Calculating for User -  vonnyc
Calculating for User -  voxemluth
Calculating for User -  voxlung
Calculating for User -  vpayne
Calculating for User -  vpimentel69
Calculating for User -  vquintana01
Calculating for User -  vreal88
Calculating for User -  vs09
Calculating for User -  vskr
Calculating for User -  vsmithtn
Calculating for User -  vstha
Calculating for User -  vtek
Calculating for User -  vtemtdisp
Calculating for User -  vtgoose
Calculat

Calculating for User -  wildbill75
Calculating for User -  wildbillkelso
Calculating for User -  wildchild23
Calculating for User -  wildclay12
Calculating for User -  wilddddkatttt80
Calculating for User -  wildernessjack
Calculating for User -  wildisde
Calculating for User -  wildlife65
Calculating for User -  wildman
Calculating for User -  wile
Calculating for User -  wileybutterfly
Calculating for User -  will
Calculating for User -  will1014
Calculating for User -  will2013
Calculating for User -  willbee
Calculating for User -  willi
Calculating for User -  william
Calculating for User -  william p
Calculating for User -  williamk
Calculating for User -  williammark
Calculating for User -  williams
Calculating for User -  williams95
Calculating for User -  williamsburg jen
Calculating for User -  williamsonk0316
Calculating for User -  willie
Calculating for User -  willow123
Calculating for User -  willowbard
Calculating for User -  willowgold
Calculating for User -  willowsmo

Calculating for User -  xpotatotitsx
Calculating for User -  xqhinc
Calculating for User -  xracer
Calculating for User -  xray
Calculating for User -  xraymom
Calculating for User -  xsbluesugar
Calculating for User -  xsnitro
Calculating for User -  xstr8edgex
Calculating for User -  xtinaa01
Calculating for User -  xtony
Calculating for User -  xxacw12xx
Calculating for User -  xxcris
Calculating for User -  xxfloridagirlxx
Calculating for User -  xxxxxxxxxxxx
Calculating for User -  xxyy
Calculating for User -  xyoung
Calculating for User -  xyzmom
Calculating for User -  xzaclee16
Calculating for User -  xzxsarahzxz
Calculating for User -  y4all
Calculating for User -  yadleyburg420
Calculating for User -  yagerbomb07
Calculating for User -  yago
Calculating for User -  yaii
Calculating for User -  yalc
Calculating for User -  yama
Calculating for User -  yamel97
Calculating for User -  yams2536
Calculating for User -  yana
Calculating for User -  yang
Calculating for User -  yang

Calculating for User -  zt313
Calculating for User -  zubb
Calculating for User -  zulaa118
Calculating for User -  zuttle
Calculating for User -  zwithanx
Calculating for User -  zxcsdfd
Calculating for User -  zxjki
Calculating for User -  zyiah4
Calculating for User -  zzdiane
Calculating for User -  zzz1127


In [21]:
## Checking for user who does not have any postive recommendations
final_user_df.loc[final_user_df.username=="repster1988"]

,product_name,recommended_pct,predicted_sentiment_sum,predicted_sentiment_count,pct_positive_review,username


In [22]:
final_user_df.loc[final_user_df.username=="1085"]

,product_name,recommended_pct,predicted_sentiment_sum,predicted_sentiment_count,pct_positive_review,username
17,Stargate (ws) (ultimate Edition) (director's C...,9.174304,180,186,96.774194,1085
7,Red (special Edition) (dvdvideo),20.229913,641,672,95.386905,1085
2,Planes: Fire Rescue (2 Discs) (includes Digita...,76.771171,1088,1143,95.188101,1085
8,Clorox Disinfecting Bathroom Cleaner,19.853915,1889,2039,92.643453,1085
15,Jason Aldean - They Don't Know,9.546500,188,204,92.156863,1085


## Conclusion - Part C



* User Based Colloborative Filtering used to generate user item ratings
* Logistic regression used to predict sentiment for review text
* Combining both, final top 5 recommended products for user is derived and pickled.
* This Pickle file would be used in Flask app 


In [23]:
import pickle

# Save to file in the current working directory
pkl_filename = "final_user_df.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(final_user_df, file)


## Deployment to Heroku

This project has been deployed onto https://sbprs-product-reco-manuel.herokuapp.com/ via https://github.com/mPAND/SBPRS